In [1]:
using BirkhoffDecomposition
using SparseArrays
using Clp 
using JuMP

In [2]:
n = 5

5

In [3]:
M = zeros(n*n,2*n);
for i=1:n
    M[(i-1)*n*n + (i-1)*n + 1 : (i-1)*n*n + (i-1)*n + n ] = ones(n,1);
end
for i=1:n
    for j=1:n
        M[n*n*n + (i-1)*n*n + (j-1)*n + i] = 1;
    end
end

A = sparse(M');
b = ones(2*n);

In [4]:
using HiGHS
model = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [5]:
@variable(model, 0 <= x[1:n*n] <= 1)
@constraint(model, A * x .== b)

10-element Vector{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 x[1] + x[2] + x[3] + x[4] + x[5] = 1
 x[6] + x[7] + x[8] + x[9] + x[10] = 1
 x[11] + x[12] + x[13] + x[14] + x[15] = 1
 x[16] + x[17] + x[18] + x[19] + x[20] = 1
 x[21] + x[22] + x[23] + x[24] + x[25] = 1
 x[1] + x[6] + x[11] + x[16] + x[21] = 1
 x[2] + x[7] + x[12] + x[17] + x[22] = 1
 x[3] + x[8] + x[13] + x[18] + x[23] = 1
 x[4] + x[9] + x[14] + x[19] + x[24] = 1
 x[5] + x[10] + x[15] + x[20] + x[25] = 1

In [6]:
struct polytope
    A;
    b;
    l;
    u;
    model;
    x;
end

In [7]:
P = polytope(A,b,0,1,model,x)

polytope(sparse([1, 6, 1, 7, 1, 8, 1, 9, 1, 10  …  5, 6, 5, 7, 5, 8, 5, 9, 5, 10], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 10, 25), [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 0, 1, A JuMP Model
Feasibility problem with:
Variables: 25
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 10 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 25 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 25 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS
Names registered in the model: x, VariableRef[x[1], x[2], x[3], x[4], x[5], x[6], x[7], x[8], x[9], x[10]  …  x[16], x[17], x[18], x[19], x[20], x[21], x[22], x[23], x[24], x[25]])

In [8]:
P.A

10×25 SparseMatrixCSC{Float64, Int64} with 50 stored entries:
⎡⠉⠉⠑⠒⠒⠤⠤⢄⣀⣀⠀⠀⠀⎤
⎢⠢⡀⠐⢄⠀⠢⡀⠐⢄⠀⠫⡉⠁⎥
⎣⠀⠈⠂⠀⠑⠀⠈⠂⠀⠑⠀⠈⠂⎦

In [9]:
c = ones(n*n)
@objective(P.model, Min, c'* P.x)

x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7] + x[8] + x[9] + x[10] + x[11] + x[12] + x[13] + x[14] + x[15] + x[16] + x[17] + x[18] + x[19] + x[20] + x[21] + x[22] + x[23] + x[24] + x[25]

In [10]:
print(model)

In [11]:
set_optimizer_attribute(model, "solver", "simplex")

optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
10 rows, 25 cols, 50 nonzeros
9 rows, 25 cols, 45 nonzeros
Presolve : Reductions: rows 9(-1); columns 25(-0); elements 45(-5)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 9(9) 0s
         13     5.0000000000e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 13
Objective value     :  5.0000000000e+00
HiGHS run time      :          0.00


In [12]:
value.(P.x)

25-element Vector{Float64}:
  0.0
  1.0
 -0.0
  0.0
  0.0
 -0.0
  0.0
  1.0
  0.0
  0.0
  0.0
 -0.0
  0.0
 -0.0
  1.0
  1.0
  0.0
  0.0
  0.0
  0.0
  0.0
  0.0
 -0.0
  1.0
  0.0